# Latent Semantic Analysis

One natural language processing (NLP) method I've always found interesting is latent semantic analysis (LSA), an early  method that uses matrix decomposition to discover unobserved "latent" semantic associations between words in different documents. In this project, I want to use this method to construct an index for a particular work. This was inspired by Cosma Shalizi's data analysis book, in which he discusses using latent semantic indexing (LSI) to construct an index in Adam Smith's "The Wealth of Nations." First, we import the modules we will need. The `gensim` library contains import modules for topic modelling, including the `LsiModel`. 


In [176]:
import PyPDF2
from gensim.models import LsiModel
from gensim import models
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords, stem_text
from gensim.parsing.preprocessing import strip_numeric
import pandas as pd
from gensim import similarities
import logging
logger = logging.getLogger("PyPDF2")
logger.setLevel(logging.ERROR)

Next, we read in the pdf data and collect the document in a list called `cor`.

In [189]:
wealth_file = 'wealthofnations2.pdf'
wealth = open(wealth_file, 'rb')
wealthReader = PyPDF2.PdfReader(wealth)

# The actual text starts on page 109, which is 111 of the pdf (which is 110 in Python as lists are indexed from 0)
first_page = 7
last_page = 784

# Collect the corpus
cor = []
for num in range(first_page, last_page):
    cor.append(wealthReader.pages[num].extract_text())

Then we construct a generator function that strips out numeric values, stems the words, and then tokenizes each document (page of the text). Once each document is processed, we construt a mapping between tokens and their integer IDs with `corpora.Dictionary` that takes an iterable. Once we have the dictionary, we filter extreme values, removing ... Finally, we construct a document-term matrix that creates a list of `(token_id, token_count)` tuples for each document.

In [191]:
def preprocessing():
    for document in cor:
        doc = remove_stopwords(strip_numeric(stem_text(document)))
        yield gensim.utils.tokenize(doc, lower=True)
        
texts = preprocessing()
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1)
doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in preprocessing()]

## Latent Indexing

How would we normally construct an index? One obvious way to do this is to look at raw word counts and peel off the top $n$ pages that most frequently mention a particular word. In this case, our word of interest is "agriculture."


In [204]:
query = 'agriculture'
query_id = dictionary.token2id.get(query)
doc_frequencies = [(doc_id, sum(count for tok_id, count in doc if tok_id == query_id)) for doc_id, doc in enumerate(doc_term_matrix)]
doc_frequencies.sort(key =lambda x: x[1], reverse=True)
doc_frequencies = doc_frequencies[:10]

for doc_id, count in doc_frequencies:
    print(f"Printed page {doc_id + 2} has the word '{query}' {count} times")

Printed page 166 has the word 'agriculture' 4 times
Printed page 126 has the word 'agriculture' 3 times
Printed page 293 has the word 'agriculture' 3 times
Printed page 300 has the word 'agriculture' 3 times
Printed page 333 has the word 'agriculture' 3 times
Printed page 6 has the word 'agriculture' 2 times
Printed page 121 has the word 'agriculture' 2 times
Printed page 154 has the word 'agriculture' 2 times
Printed page 551 has the word 'agriculture' 2 times
Printed page 761 has the word 'agriculture' 2 times


This doesn't work in general, because it will only include pages that have the word "agriculture" and will not include pages that discuss topics related to agriculture but don't use the word itself (Shalizi 2016, p. 383). Instead, we will construct a word-document matrix that we can use to find latent relationships between words and documents in a corpus. The idea is that we can discover these relationships and use them to find documents that are most related to a particular word. 

In the original LSI paper, the authors use a pure "count" matrix that measures the number of times each word appears in each document (which in this project is a page of the text). Instead, we  use a term frequency-inverse document frequency (tf-idf) matrix, which in addition to counts incorporates how frequently a word appears across the corpus. We compute the tf-idf matrix using our document-term matrix with `TfidfModel`, which computes the tf-idf weights for each token. We then transform the full corpus using our model.



In [193]:
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

For an $m \times n$ matrix $A$, we can decompose $A$ as:

\begin{equation}
A = U \Sigma V^T
\end{equation}

where $U$ and $V$ are orthogonal matrices and $\Sigma$ is a diagonal matrix. The columns of $U$ are the left singular vectors of $A$ and are the eigenvectors of $AA^T$. Similarly, the columns are the right singular vectors of $A$, or the eigenvectors of $A^TA$. These two matrices can be used to eigendecompose the matrices $AA^T$ or $A^TA$. The idea behind LSI is that, once we have the SVD decomposition of $X$, we can find a lower dimensional representation of the matrix by truncating the singular values to only keep the $k$ largest ones. It is a well-known theorem that this produces the best rank-$k$ approximation to the original matrix (measured in terms of the Frobenius norm). 

We rarely have a clear idea of what $k$ should be in any practical application. As with principal component analysis, this is primarily a trial and error process. The original LSI paper recommends using between 50-100 factors (Deerwester et. al. 1990, p. 7). In this case, we use 50 singular vectors to truncate the SVD. The module `sklearn` has a fast truncated SVD decomposition in the `TruncatedSVD` class.

Below, we instantiate the `LsiModel` with 50 topics (the latent factors we discussed in the previous paragraph). Then we take the query word (or "pseudo-document" in the language of (Deerwester et. al. 1990)) and transform it into the $k$-dimensional space of latent concepts. Once we have done that, we calculate the similarity between every document to every other document, represented in this space. Finally, we can see which documents are most similar to our query. We print the top 20 most similar documents.

In [207]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=50)
vec_bow = dictionary.doc2bow(query.lower().split())

vec_lsi = lsi[vec_bow]  # convert the query to LSI space
index = similarities.MatrixSimilarity(lsi[doc_term_matrix])
unsorted_similarity = index[vec_lsi]
sorted_similarity = sorted(enumerate(unsorted_similarity), key=lambda x: x[1], reverse=True)
for index, similarity in sorted_similarity[:20]:
    print(similarity, index)

0.557081 298
0.541889 332
0.5403838 331
0.5300852 302
0.5190811 320
0.514124 319
0.4811794 164
0.47760132 321
0.47642392 291
0.46523505 289
0.4324791 290
0.42376462 322
0.41840863 544
0.3964745 760
0.39538434 5
0.39408797 548
0.39388704 551
0.39342406 273
0.38387808 300
0.37645245 312


Let's look at the most similar documents to our query:

In [209]:
cor[332]

'340The Wealth of Nations\nThe foreign commerce of Spain and Portual to the other parts\nof Europe, though chiefly carried on in foreign ships, is very con-\nsiderable. That to their colonies is carried on in their own, and is\nmuch greater, on account of the great riches and extent of those\ncolonies. But it has never introduced any considerable manufac-\ntures for distant sale into either of those countries, and the greater\npart of both still remains uncultivated. The foreign commerce of\nPortugal is of older standing than that of any great country in\nEurope, except Italy.\nItaly is the only great country of Europe which seems to have\nbeen cultivated and improved in every part, by means of foreign\ncommerce and manufactures for distant sale. Before the invasion\nof Charles VIII., Italy, according to Guicciardini, was cultivated\nnot less in the most mountainous and barren parts of the country,\nthan in the plainest and most fertile. The advantageous situation\nof the country, and 

## References

- Deerwester, S., Dumais, S. T., Furnas, G. W., Landauer, T. K., & Harshman, R. (1990). Indexing by latent semantic analysis. Journal of the American society for information science, 41(6), 391-407.
- Smith, A. (1970). The Wealth of Nations Books I—III. Pelican Books.
- Shalizi, C. R. (2016). Advanced data analysis from an elementary point of view. 2013. URL http://www.stat.cmu.edu/~cshalizi/ADAfaEPoV.